In [1]:
# Useful starting lines
%matplotlib inline

import numpy as np
import scipy
import scipy.io
import scipy.sparse as sp
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

In [2]:
from helpers import *
from cost import *
crossValidation = False
if crossValidation:
    trainDF, testDF = crossValidationSets(0.7)
else:
    trainDF, testDF = realDataSets() # Kaggle datasets (with no predictions in valuesTest)

print(trainDF.head())
print(testDF.head())

number of items: 10000, number of users: 1000
number of items: 10000, number of users: 1000
   User  Movie  Prediction
0     0     43         4.0
1     0     60         3.0
2     0     66         4.0
3     0     71         3.0
4     0     85         5.0
   User  Movie
0     0     36
1     0     72
2     0    155
3     0    159
4     0    247


In [3]:
from predictionAlgorithms import *

rankALS = 6
numIterationsALS = 20
lambdaALS = 0.07585775750291851

rankALS = 8
lambdaALS = 0.081

predictionsUserMean = baseline_user_mean(trainDF, testDF)
predictionsItemMean = baseline_item_mean(trainDF, testDF)
#predictionsALS = ALSPyspark(trainDF, testDF, rankALS, lambdaALS, numIterationsALS)
predictionsALS = ALSPyspark(trainDF, testDF, rankALS, lambdaALS, numIterationsALS)

In [4]:
#print(predictionsALS.Prediction.values)
print(predictionsALS.head())
print(testDF.head())

   _1   _2        _3
0   0   36  3.191471
1   0   72  3.046421
2   0  155  3.696488
3   0  159  3.279599
4   0  247  3.516067
   User  Movie
0     0     36
1     0     72
2     0    155
3     0    159
4     0    247


In [ ]:
print(predictionsALS.head())
testDF.sort_values(['User', 'Movie'], ascending=[1,1], inplace=True)
print(testDF.head())
#print(predictionsALS.Prediction)
#print(testDF.Prediction)
print(compute_cost(predictionsALS._3, trainDF.Prediction.values))
if crossValidation:
    #print(np.mean((predictionsALS._3-testDF.Prediction.values)**2))
    print(compute_cost(predictionsALS._3, testDF.Prediction.values))
    #print(np.mean((predictionsALSMartin._3.values-testDF.Prediction)**2))

In [ ]:
wUserMean = 0.0204081632653
wItemMean = 0.020842379505
wALS = 0.979848228802
#predictions = (wUserMean*predictionsUserMean + wItemMean*predictionsItemMean + wALS*predictionsALS['Prediction']) / (wUserMean + wItemMean + wALS)
predictions = (wUserMean*predictionsUserMean + wItemMean*predictionsItemMean + wALS*predictionsALS._3.values) / (wUserMean + wItemMean + wALS)
print(predictionsUserMean[0])
print(predictionsItemMean[0])
#print(predictionsALS.Prediction.values[0])
print(predictionsALS._3.values[0])
print(predictions[0])

In [ ]:
from cv import *
wUser, wItem, wALS, loss = optimize_weights(predictionsUserMean, predictionsItemMean, predictionsALS._3.values, testDF.Prediction.values)
print(wUser)
print(wItem)
print(wALS)
print(loss)

In [ ]:
from helpers import *
make_submission(predictions)

In [ ]:
predictionsALS

In [ ]:
from predictionAlgorithms import *
predictionsALS = ALSPyspark(trainDF, testDF)

In [ ]:
predictionsALS.columns = ['User', 'Movie', 'Prediction']
predictionsALS.sort_values(['User', 'Movie'], ascending=[1,1], inplace=True)
predictionsALS.head()

In [ ]:
predictions = 0.25*predictionsUserMean+0.25*predictionsItemMean + 0.5*predictionsALS['Prediction']

In [ ]:
make_submission(predictions)